In [ ]:
import data_analysis.data_processing as dp
import data_analysis.data_learning as dl

# reload module to bypass caching
import importlib
importlib.reload(dp)
importlib.reload(dl)

# paths
test_window_path_home = dp.Path(r'\\?\C:\Users\jannis\Documents\HSP_IDS\Material\Aktuell\2025-02-17_11-14-33_192.168.1.0-normal_1\1554220324.748197-1554220354.748197') # treat it as a long path to avoid path length issues on windows
test_window_path_remote = dp.Path(r'/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff/2025-03-04_00-03-20_192.168.1.0-normal_DDoS_1/1556203726.876922-1556203756.876922')

attack_data_set_path = dp.Path(r'/home/hsp252/nas_mount/hunter.ids.data/hunter.ids.preprocessor/processed_dataframes/angriff')
ddos_test_path_parquet = dp.Path(r'/home/hsp252/Development/DDoS')

start = dp.time.time()
test_ddos_parquet = dp.pd.read_parquet(ddos_test_path_parquet / 'intrusion_normal_DDoS_10_final.parquet')
end = dp.time.time()
print('time to read parquet ddos file: ' + str(end - start) + 's')

feature_vectors = dp.pd.DataFrame()

# group df by window time key and iterate so not to mix up 30s windows
learning_dataset = dp.pd.DataFrame()
test_ddos_parquet_window_time_key = test_ddos_parquet.groupby('window_time_key')

for i, tsw in test_ddos_parquet_window_time_key:

    # get connections
    test_ddos_parquet_conn_src = tsw.reset_index().groupby('conn_src_ip')
    test_ddos_parquet_conn_dst = tsw.reset_index().groupby('conn_dst_ip')

    for key in test_ddos_parquet_conn_src.groups.keys() & test_ddos_parquet_conn_dst.groups.keys():

        # get all connections for a host
        host = dp.pd.DataFrame()
        host = dp.pd.concat([host, test_ddos_parquet_conn_src.get_group(key)], axis=0, ignore_index=True)
        host = dp.pd.concat([host, test_ddos_parquet_conn_dst.get_group(key)], axis=0, ignore_index=True)
        host.reset_index().drop('index', axis=1)

        # get average connection values
        bool_columns = host.select_dtypes(include='bool').columns
        host[bool_columns] = host[bool_columns].astype(float)
        mean_host = host.select_dtypes(include='number').mean().fillna(0)

        # groupy by connection and mean
        host_mean_connections = host.groupby(['conn_protocol', 'conn_src_ip', 'conn_dst_ip']) # this is key of a connection across 10s windows
        feature_vector = dp.pd.DataFrame()
        
        for i, group in host_mean_connections:
            group = group.select_dtypes(include='number')
            group = group.fillna(mean_host).mean().to_frame().transpose()
            feature_vector = dp.pd.concat([feature_vector, group])
        
        # create feature vector
        feature_vector = feature_vector.median().to_frame().transpose()
        feature_vectors = dp.pd.concat([feature_vectors, feature_vector])

        #break
    #break
    


    
    # put that into a dataset for RFC
    # for non-DDoS stuff: acquire through 30s-windows, hosts and connections, do mean and median and add to dataset for training RFC
    # check the least amount of 30s windows in attack classes and take this amount of 30s windows for now


# # s1
# if 'gq_mean_queue_length' in columns:
#     print(columns.index('gq_mean_queue_length')) # 159
# if 'time_window_end' in columns:
#     print(columns.index('time_window_end') + '\n') # 

# columns = test_ddos_parquet.columns.to_list()
# print(columns)
# #  # s2
# if 'sq_identifier' in columns:
#     print(columns.index('sq_identifier')) # 
# if 'is_most_active_host_based_on_packets' in columns:
#     print(columns.index('is_most_active_host_based_on_packets') + '\n') # 

# # s3
# if 'sq_identifier' in columns:
#     print(columns.index('sq_identifier')) # 
# if 'connection_tcp_udp_other_ratio' in columns:
#     print(columns.index('connection_tcp_udp_other_ratio') + '\n') # 

# # connection features
# if 'selected_queue' in columns:
#     print(columns.index('selected_queue')) # 0
# if 'conn_count_of_zeros_dst' in columns:
#     print(columns.index('conn_count_of_zeros_dst')) # 114


# thirty_second_windows_paths = dp.getThirtySecondWindowPaths(attack_data_set_path)
# sample = thirty_second_windows_paths.groupby("type").sample(n=10, random_state=42).reset_index(drop=True)

# # save each 30s window in a file
# for index, row in sample.iterrows():
#     print(row['path'])
#     tsw = dp.ThirtySecondWindow(dp.Path(row['path']))

#     type = row["type"]
#     name = dp.Path(row['path']).name
#     data = {}

#     save_path = f'attack/{type}/{name}'
#     print(dp.pp.save(save_path, tsw))
#     print("done}\n")
#     break

# # # create feature vectors for sample
# # X = []
# # Y = []

# # for file in dp.os.scandir('attack'):
# #     thirty_second_window : dp.ThirtySecondWindow = dp.pp.load(f'attack/{file}')
# #     type = dp.Path(file).parts(-2)

# #     X.append(thirty_second_window)

time to read parquet ddos file: 10.88927698135376s
(3, 160)
   conn_src_port  conn_dst_port  host_ct_dst_addresses  host_ct_src_ports  \
0        49255.0         1900.0                    3.2              82.85   

   host_ct_dst_ports  host_ct_protocols  host_ct_syn  host_ct_ack  \
0              82.35                4.0        83.05        252.8   

   host_ct_fin  host_ct_cwr  ...  gq_highest_priority  gq_lowest_priority  \
0         29.8          0.0  ...                  0.0                 0.0   

   gq_host_count_diff  gq_protocol_queue_count_diff  \
0                10.0                           2.0   

   protocol_queue_count_layer_4_diff  protocol_queue_count_layer_5_plus_diff  \
0                                0.0                                     0.0   

   gq_growth_rates  gq_growth_rates_percentage  gq_mean_growth_rate  \
0              0.0                         0.0                  0.0   

   gq_median_growth_rate  
0                    0.0  

[1 rows x 159 columns